<a href="https://colab.research.google.com/github/Reubenpt123/DAV/blob/main/Data_Cleaning_DAV_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Connect to google drive and import necessary libraries.

In [13]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, r2_score

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Change working directory.

In [14]:
cd "/content/drive/My Drive/Colab Notebooks/DAV Project/Cefas_smartbuoy"

/content/drive/My Drive/Colab Notebooks/DAV Project/Cefas_smartbuoy


Import files from drive.


In [15]:
ls

new_data/                WESTGAB2.csv     WESTGAB.csv     WG2O2SAT.csv
oxygen_saturation_data/  WESTGAB2PAR.csv  WESTGABPAR.csv


Need to deal with encoding errors in order for file to open file. The causes are in the units column and can be ignored. Then we use the dateTime column to be of the date type and use this to index our files.

In [16]:
westgab = pd.read_csv("WESTGAB.csv", parse_dates = ["dateTime"], encoding_errors = "ignore")
westgab_2 = pd.read_csv("./new_data/WESTGAB_2.csv", encoding_errors = "ignore")
westgab.set_index("dateTime", inplace = True)
westgab_2.set_index("dateTime", inplace = True)
westgabpar = pd.read_csv("WESTGABPAR.csv", encoding_errors = "ignore")
westgabpar.set_index("dateTime", inplace = True)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (12) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


Now we extract the data associated with seperate parameters into seperate data frames before re-combining based on date and making sure that we've got our datatypes set.

In [17]:
westgab_temp = westgab[westgab["par"] == "TEMP"]
westgab_temp = westgab_temp["value"]
westgab_temp.name = "temp"
westgab_sal = westgab[westgab["par"] == "SAL"]
westgab_sal = westgab_sal["value"]
westgab_sal.name = "sal"
westgab_ftu = westgab[westgab["par"] == "FTU"]
westgab_ftu = westgab_ftu["value"]
westgab_ftu.name = "ftu"
westgab_fluors = westgab[westgab["par"] == "FLUORS"]
westgab_fluors = westgab_fluors["value"]
westgab_fluors.name = "fluors"
westgab_o2 = westgab[westgab["par"] == "O2CONC"]
westgab_o2 = westgab_o2["value"]
westgab_o2.name = "o2"

westgab_kd = westgab_2[westgab_2["par"] == "Kd"]
westgab_kd = westgab_kd["value"]
westgab_kd.name = "kd"
westgab_tox = westgab_2[westgab_2["par"] == "TOXN"]
westgab_tox = westgab_tox["value"]
westgab_tox.name = "tox"
westgab_sil = westgab_2[westgab_2["par"] == "SILICA"]
westgab_sil = westgab_sil["value"]
westgab_sil.name = "sil"

westgab_par = westgabpar["value"]
westgab_par.name = "par"


merge = pd.merge(westgab_temp, westgab_sal, how = 'outer', left_index = True, right_index = True)
merge = pd.merge(merge, westgab_ftu, how = 'outer', left_index = True, right_index = True)
merge = pd.merge(merge, westgab_fluors, how = 'outer', left_index = True, right_index = True)
merge = pd.merge(merge, westgab_o2, how = 'outer', left_index = True, right_index = True)
merge = pd.merge(merge, westgab_kd, how = 'outer', left_index = True, right_index = True)
merge = pd.merge(merge, westgab_sil, how = 'outer', left_index = True, right_index = True)
merge = pd.merge(merge, westgab_par, how = 'outer', left_index = True, right_index = True)

merge = merge.apply(pd.to_numeric, errors = 'raise')
merge.to_csv("clean_data.csv", sep = ",", index = "False")

In [18]:
df = pd.read_csv("clean_data.csv", parse_dates = ["dateTime"])
df

,dateTime,temp,sal,ftu,fluors,o2,kd,sil,par
0,2002-08-28 13:15:00,18.78,34.691,6.42,0.62,NaN,NaN,NaN,NaN
1,2002-08-28 13:30:00,18.77,34.666,6.56,0.63,NaN,NaN,NaN,NaN
2,2002-08-28 13:45:00,18.75,34.653,5.69,0.66,NaN,NaN,NaN,NaN
3,2002-08-28 14:00:00,18.75,34.635,5.50,0.69,NaN,NaN,NaN,NaN
4,2002-08-28 14:15:00,18.74,34.628,4.93,0.75,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
493326,2016-05-10 04:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.8
493327,2016-05-10 04:30:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.2
493328,2016-05-10 05:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.7
493329,2016-05-10 05:30:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,51.6
